In [ ]:
from utils import *
import getpass
from landsatxplore.earthexplorer import EarthExplorer
import tarfile
import boto3

In [ ]:
aws_session = rasterio.session.AWSSession(boto3.Session(), requester_pays=True)
# aws_session = rasterio.session.AWSSession(boto3.Session())

In [ ]:
bbox = read_kml_polygon("data/inputs/LANDSAT_8_127111/WA.kml")[1]
bbox = resize_bbox(bbox, 0.1)

In [ ]:
user = getpass.getpass("Enter your username?")
password = getpass.getpass("Enter your password?")

In [ ]:
start_date = "2023-01-01T00:00:00"
end_date: str = "2025-01-01T00:00:00"
platform = "LANDSAT_8"
#   query = get_landsat_search_query(
#   [67.45, -72.55, 67.55, -72.45],
#   start_date=start_date,
#   end_date=end_date,
#   platform=platform
# ) # Amery ice shelf bed rock
# query = get_landsat_search_query(
#     [69.2, -68.1, 69.4, -67.9],
#     start_date=start_date,
#     end_date=end_date,
#     platform=platform,
#     cloud_cover=50,
# )  # Top of Amery ice shelf
server_url = "https://landsatlook.usgs.gov/stac-server/search"
query = get_landsat_search_query(
    bbox,
    start_date=start_date,
    end_date=end_date,
    platform=platform,
)  # WA sand dunes
features = query_stac_server(query, server_url)
feat_dict = find_landsat_scenes_dict(features)

In [ ]:
s = pd.Series(list(feat_dict.keys())).astype("str")
g = [s.split("_")[3][0:6] for s in list(feat_dict.keys())]
feat_dict = {id: feat_dict[id] for id in [el[1].iloc[0] for el in list(s.groupby(g))]}

In [ ]:
aws_reds = [feat_dict[k]["red"][1] for k in feat_dict]
bounds_dict = {}
for r in aws_reds:
    id = os.path.basename(r).split("_")[2]
    meta = stream_scene_from_aws(r, aws_session, metadata_only=True)[1]
    bounds_dict[id] = {"bounds": meta["bounds"], "crs": meta["crs"].data}

In [ ]:
for key in bounds_dict:
    bounds = bounds_dict[key]["bounds"]
    crs = bounds_dict[key]["crs"]
    bottom_utm = UTM(bounds.left, bounds.bottom)
    top_utm = UTM(bounds.right, bounds.top)
    bottom_lla = UTMtoLLA(bottom_utm, crs)
    top_lla = UTMtoLLA(top_utm, crs)
    print(key, bottom_lla, top_lla)

In [ ]:
path_num = "108074"
required_keys = [k for k in list(feat_dict.keys()) if path_num in k]
required_scenes = [feat_dict[k]["scene_id"] for k in required_keys]
len(required_scenes)

In [ ]:
scene_index = 13

r_url = feat_dict[required_keys[scene_index]]["red"][0]
g_url = feat_dict[required_keys[scene_index]]["green"][0]
b_url = feat_dict[required_keys[scene_index]]["blue"][0]

r_aws = feat_dict[required_keys[scene_index]]["red"][1]
g_aws = feat_dict[required_keys[scene_index]]["green"][1]
b_aws = feat_dict[required_keys[scene_index]]["blue"][1]

In [ ]:
r_img, _ = stream_scene_from_aws(r_aws, aws_session)
g_img, _ = stream_scene_from_aws(g_aws, aws_session)
b_img, _ = stream_scene_from_aws(b_aws, aws_session)

In [ ]:
rgb = apply_gamma(cv.merge([r_img[0, :, :], g_img[0, :, :], b_img[0, :, :]]), 1.0, True)
plt.imshow(rgb)

In [ ]:
for key in required_keys:
    print(f"Now downloading {key}")
    r_url = feat_dict[key]["red"][1]
    g_url = feat_dict[key]["green"][1]
    b_url = feat_dict[key]["blue"][1]
    output_dir = f"data/inputs/{platform}_{path_num}/{key}"
    os.makedirs(output_dir, exist_ok=True)
    r_output = os.path.join(output_dir, os.path.basename(r_url))
    g_output = os.path.join(output_dir, os.path.basename(g_url))
    b_output = os.path.join(output_dir, os.path.basename(b_url))
    r_img, r_meta = stream_scene_from_aws(r_url, aws_session)
    g_img, g_meta = stream_scene_from_aws(g_url, aws_session)
    b_img, b_meta = stream_scene_from_aws(b_url, aws_session)

    imgs = [r_img, g_img, b_img]
    outputs = [r_output, g_output, b_output]
    metas = [r_meta, g_meta, b_meta]
    for i, img in enumerate(imgs):
        with rasterio.open(outputs[i], "w", **metas[i]["profile"]) as ds:
            ds.write(img[0, :, :], 1)

In [ ]:
# ee = EarthExplorer(user, password)
# for scene in required_scenes:
#     output_dir = f"data/inputs/{platform}_{path_num}"
#     os.makedirs(f"data/inputs/{platform}_{path_num}", exist_ok=True)
#     print(f"Downloading {scene}")
#     ee.download(scene, output_dir)
# ee.logout()

In [ ]:
# tar_files = [
#     f for f in glob.glob(f"data/inputs/{platform}_{path_num}/**") if f.endswith(".tar")
# ]
# for tar_file in tar_files:
#     extraction_dir = os.path.join(
#         output_dir, os.path.splitext(os.path.basename(tar_file))[0]
#     )
#     os.makedirs(extraction_dir, exist_ok=True)
#     with tarfile.open(tar_file) as tf:
#         tf.extractall(extraction_dir)

In [ ]:
red = f"data/inputs/LANDSAT_8_132108/LC08_L2SR_132108_20241117_20241126_02_T2_SR/LC08_L2SR_132108_20241117_20241126_02_T2_SR_B4.TIF"
green = f"data/inputs/LANDSAT_8_132108/LC08_L2SR_132108_20241117_20241126_02_T2_SR/LC08_L2SR_132108_20241117_20241126_02_T2_SR_B3.TIF"
blue = f"data/inputs/LANDSAT_8_132108/LC08_L2SR_132108_20241117_20241126_02_T2_SR/LC08_L2SR_132108_20241117_20241126_02_T2_SR_B2.TIF"

redf = flip_img(rasterio.open(red).read())[:, :, 0]
redf = apply_gamma(redf, 1.0, True)

greenf = flip_img(rasterio.open(green).read())[:, :, 0]
greenf = apply_gamma(greenf, 1.0, True)

bluef = flip_img(rasterio.open(blue).read())[:, :, 0]
bluef = apply_gamma(bluef, 1.0, True)


img = cv.merge([redf, bluef, greenf])
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
true_color_dir = f"data/inputs/{platform}_{path_num}/true_color"
os.makedirs(true_color_dir, exist_ok=True)
scenes_dirs = [
    dir for dir in glob.glob(f"data/inputs/{platform}_{path_num}/**") if "_T1" in dir
]
for dir in scenes_dirs:
    print(f"Now processing {os.path.basename(dir)}.")
    files = glob.glob(f"{dir}/**")
    r_band = list(filter(lambda f: f.endswith("_B4.TIF"), files))[0]
    g_band = list(filter(lambda f: f.endswith("_B3.TIF"), files))[0]
    b_band = list(filter(lambda f: f.endswith("_B2.TIF"), files))[0]
    true_bands = [r_band, g_band, b_band]
    make_landsat_true_color_scene(
        true_bands, f"{os.path.join(true_color_dir, os.path.basename(dir))}_TC.TIF"
    )

In [ ]:
true_color_ds_dir = f"data/inputs/{platform}_{path_num}/true_color_ds"
os.makedirs(true_color_ds_dir, exist_ok=True)
tc_files = glob.glob(f"data/inputs/{platform}_{path_num}/true_color/**")
for tcf in tc_files:
    downsample_dataset(tcf, 0.2, os.path.join(true_color_ds_dir, os.path.basename(tcf)))